## Alpha Vantage - PE Ratio Example

In [4]:
import os # for referencing environment variables
from dotenv import load_dotenv # for loading environment variables 
import requests

In [5]:
load_dotenv() # load environment variables from .env file
alpha_vantage_api_key = os.getenv('ALPHA_VANTAGE_API_KEY') # Get your API key from https://www.alphavantage.co/support/#api-key

In [6]:
ticker = 'AAPL'
url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={alpha_vantage_api_key}'
r = requests.get(url)
data = r.json()
pe_ratio = float(data['PERatio'])
pe_ratio

35.85